In [ ]:
from matplotlib import pyplot as plt
import numpy as np

from firedrake import *



In [ ]:
mask = np.load("data/filled/1.npy")

# mask[:, -1] = 0

mask

In [ ]:
n = mask.shape[0]

mesh = PeriodicUnitSquareMesh(nx=n, ny=n)

# triplot(mesh)

In [ ]:
WX = FunctionSpace(mesh, "WXRobH3NC", degree=7)

V = FunctionSpace(mesh, "CG", 1)

In [ ]:
m = V.ufl_domain()
W = VectorFunctionSpace(m, V.ufl_element())

X = interpolate(m.coordinates, W)

mesh_coords = X.dat.data

In [ ]:
def create_grid(n):
    # assumes periodic mesh
    return np.stack(
        np.meshgrid(np.linspace(1/n, 1., n), np.linspace(1/n, 1., n)),
        axis=-1
)


def get_nonzero_indices(grid_ones, mesh_coords):
    indices = []
    for k, coords in enumerate(grid_ones):
        print(k)
        for index, mesh_coords_ in enumerate(mesh_coords):
            if np.allclose(coords, mesh_coords_):
                indices.append(index)
                break
        else:
            raise ValueError("Something went wrong")
            
    return indices
            
    
def mask_to_vector(V, mask):
    # assume 
    m = V.ufl_domain()
    W = VectorFunctionSpace(m, V.ufl_element())

    X = interpolate(m.coordinates, W)

    mesh_coords = X.dat.data
    n = int(np.sqrt(V.mesh().size(0)))
    
    grid = create_grid(n)
    grid_ones = grid[np.nonzero(mask)]
    print(len(grid_ones))
    
    indices = get_nonzero_indices(grid_ones, mesh_coords)
    
    data = np.zeros(mesh_coords.shape[0])
    
    data[indices] = 1.
    
    return data

        

In [ ]:
chi = Function(V)

data = mask_to_vector(V, mask)

In [ ]:
# chi = Function(V)
# data = mask_to_vector(V, mask)

chi.dat.data[:] = data.flatten()

tricontourf(chi)



In [ ]:
v, u = TrialFunction(WX), TestFunction(WX)

In [ ]:
# m = V.ufl_domain()
# W = VectorFunctionSpace(m, V.ufl_element())

# X = interpolate(m.coordinates, W)


# def create_square(coords, x_min=0.4, x_max=0.6, y_min=0.4, y_max=0.6):
#     A = np.zeros(coords.shape[0])
    
#     idx = np.logical_and(
#         np.logical_and(
#             coords[:, 0] > x_min, 
#             coords[:, 0] < x_max,
#         ),
#         np.logical_and(
#         coords[:, 1] > y_min,
#         coords[:, 1] < y_max,
#         )
#     )

#     A[idx] = 1.
    
#     return A



# chi = Function(V)
# chi.dat.data[:] = create_square(X.dat.data_ro)

# tricontourf(chi)

In [ ]:
# x, y = SpatialCoordinate(mesh)
# # expr = cos(2*pi*x) + sin(6*pi*x)
# # expr = cos(2*pi*x)
# expr = cos(2*pi*x) + sin(2*pi*y)

# chi = interpolate(expr, V)

# tricontourf(chi)

In [ ]:
a = inner(v, u)*dx - (
    -inner(Dx(Dx(Dx(v, 0), 0), 0), Dx(Dx(Dx(u, 0), 0), 0))
    - 3 * inner(Dx(Dx(Dx(v, 1), 0), 0), Dx(Dx(Dx(u, 1), 0), 0))
    - 3 * inner(Dx(Dx(Dx(v, 0), 1), 1), Dx(Dx(Dx(u, 0), 1), 1))
    - inner(Dx(Dx(Dx(v, 1), 1), 1), Dx(Dx(Dx(u, 1), 1), 1))
)*dx

L = inner(chi, u)*dx

In [ ]:
res = Function(WX, name="field")

solve(a == L, res)


In [ ]:
res.dat.data.sum()

In [ ]:
l = tricontourf(res)
plt.colorbar(l)


In [ ]:
chi.dat.data_ro

In [ ]:
x, y = SpatialCoordinate(mesh)
# expr = cos(2*pi*x)* 1 / (1 - 3*2*pi + 3*(2*pi)**2 - (2*pi)**3)
# expr = -cos(2*pi*x)* 1 / ((2*pi)**6)
expr = -expr /  ((2*pi)**6)


sol = Function(V, name="field")
sol.interpolate(expr)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    
l = tricontourf(res, axes=ax[0])
plt.colorbar(l)

# l2 = tricontourf(sol, axes=ax[1])
# plt.colorbar(l2)

In [ ]:
outfile = File("output.pvd", target_degree=1)
# outfile = File("output.pvd")


outfile.write(res)

In [ ]:
outfile = File("output_expected.pvd", target_degree=1)
# outfile = File("output_expected.pvd")

outfile.write(sol)

In [ ]:
type(sol)